In [14]:
import time

import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import os
from time import sleep

In [2]:
pageUrls = ["https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza"]
pageUrls += ["https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=" + str(i) for i in range(2, 81)]
pageUrls

['https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=2',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=3',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=4',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=5',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=6',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=7',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=8',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=9',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=10',
 'https://www.immobiliare.it/en/affitto-appartamenti/milano/?criterio=rilevanza&pag=11',
 'https://www.immobiliare.it/en/affi

In [3]:
def urlToSoup(url, *args):
    html_content = requests.get(url=url).text
    if args :
        return BeautifulSoup(html_content, 'lxml', parse_only=args[0])
    return BeautifulSoup(html_content , 'lxml')

In [4]:
# Creation of a list containing all the pages transformed into treatable elements
pages_soup_list = [urlToSoup(pageUrls[i]) for i in range(len(pageUrls))]

<!DOCTYPE html>
<html lang="en" prefix=""><head><link href="https://s1.immobiliare.it" rel="preconnect"/><link href="https://pwm.im-cdn.it" rel="preconnect"/><link href="https://crt.im-cdn.it" rel="preconnect"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="#2b6481" name="msapplication-TileColor"/><meta content="#2b6481" name="theme-color"/><meta content="Immobiliare.it" name="apple-mobile-web-app-title"/><meta content="Immobiliare.it" name="application-name"/><link href="https://s1.immobiliare.it/_next/static/media/apple-touch-icon.6eaaec1a.png" rel="apple-touch-icon"/><link href="https://s1.immobiliare.it/_next/static/media/favicon-32x32.c18ee3c9.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://s1.immobiliare.it/_next/static/media/favicon-16x16.b6a5d8cf.png" rel="icon" sizes="16x16" type="image/png"/><link color="#2b6481" href="https://s1.immobiliare.it/_next/static/media/safari-pinned-tab.ee8311f7ab52c53c.svg" rel="m

In [18]:
appartment_URLs = list()
size_page_soup_list = len(pages_soup_list)
for i in range(len(pages_soup_list)):
    # We retrieve all the appartment urls from the current page
    appartment_raw_URLs = pages_soup_list[i].find_all('a', {"class": "in-reListCard__title is-spaced"})
    appartment_raw_URLs += pages_soup_list[i].find_all('a', {"class": "in-reListCard__title"})
    appartment_raw_URLs = list(dict.fromkeys(appartment_raw_URLs))
    # For each url we only get the link
    print(len(appartment_raw_URLs), i)
    for url in appartment_raw_URLs:
        appartment_URLs.append(url.get('href'))
    # print(f'{size_page_soup_list - i} pages remaining')

25 0
25 1
25 2
25 3
25 4
25 5
25 6
25 7
25 8
25 9
25 10
25 11
25 12
25 13
25 14
25 15
25 16
25 17
25 18
25 19
25 20
25 21
25 22
25 23
25 24
25 25
25 26
25 27
25 28
25 29
25 30
25 31
25 32
25 33
25 34
25 35
25 36
25 37
25 38
25 39
25 40
25 41
25 42
25 43
25 44
25 45
25 46
25 47
25 48
25 49
25 50
25 51
25 52
25 53
25 54
25 55
25 56
25 57
25 58
25 59
25 60
25 61
25 62
25 63
25 64
25 65
25 66
25 67
25 68
25 69
25 70
25 71
25 72
25 73
25 74
25 75
25 76
25 77
25 78
25 79


In [20]:
print(len(appartment_URLs))

2000


In [21]:
def build_appartment_characteristics(url) :
    appartment_soup = urlToSoup(url)
    appartment_information = dict()
    
    # We create the appartment address before the loop since it is not linked to the appartment characteristics in the HTML structure
    try :
        address_components = appartment_soup.find('a', {"class" : "in-titleBlock__link"})
    except AttributeError :
        return
    appartment_address = list()
    try :
        city_address = address_components.find_next('span')
        district_address = city_address.find_next('span')
        street_address = district_address.find_next('span')
        appartment_address = [city_address.text, street_address.text, district_address.text]
    except AttributeError :
        appartment_address = [pd.NA, pd.NA, pd.NA]
    
    appartment_information['Address'] = appartment_address
    # We retrieve all the tags 'dl' which represents the beginning of a dictionnary like structure in the HTML code (key associated with a value)
    try :
        appartment_characteristics = appartment_soup.find_all('dl', {"class" : "in-realEstateFeatures__list"})
    except AttributeError :
        return
    for appartment_characteristic in appartment_characteristics:
        # We find the name of the characteristic through the previous sibling node
        characteristic_name = appartment_characteristic.find_previous_sibling("div").text
        # To get all the features (names and values) we build two list by comprehension
        try :
            feature_values = [feature.text for feature in appartment_characteristic.find_all('dd', {"class" : "in-realEstateFeatures__value"})]
            feature_names = [feature.text for feature in appartment_characteristic.find_all('dt', {"class" : "in-realEstateFeatures__title"})]
        except AttributeError :
            feature_names = [pd.NA]
            feature_values = [pd.NA]
        dict_features = dict()
        # We then build the dictionnary with the values found
        for feature_name, feature_value in zip(feature_names, feature_values):
            dict_features[feature_name] = feature_value
        appartment_information[characteristic_name] = dict_features
    # We now have a fully complete dictionnary that contains all the information of one appartment
    return appartment_information

In [22]:
appartments = dict()
totalApparmentsLeft = len(appartment_URLs)
for i, url in enumerate(appartment_URLs) :
    appartments[i] = build_appartment_characteristics(url)
    print(f'Appartements left to treat : {totalApparmentsLeft - i}')
    
# 28m53s processing time

Appartements left to treat : 2000
Appartements left to treat : 1999
Appartements left to treat : 1998
Appartements left to treat : 1997
Appartements left to treat : 1996
Appartements left to treat : 1995
Appartements left to treat : 1994
Appartements left to treat : 1993
Appartements left to treat : 1992
Appartements left to treat : 1991
Appartements left to treat : 1990
Appartements left to treat : 1989
Appartements left to treat : 1988
Appartements left to treat : 1987
Appartements left to treat : 1986
Appartements left to treat : 1985
Appartements left to treat : 1984
Appartements left to treat : 1983
Appartements left to treat : 1982
Appartements left to treat : 1981
Appartements left to treat : 1980
Appartements left to treat : 1979
Appartements left to treat : 1978
Appartements left to treat : 1977
Appartements left to treat : 1976
Appartements left to treat : 1975
Appartements left to treat : 1974
Appartements left to treat : 1973
Appartements left to treat : 1972
Appartements l

In [19]:
print(appartment_URLs)

['https://www.immobiliare.it/en/annunci/109715669/', 'https://www.immobiliare.it/en/annunci/109718749/', 'https://www.immobiliare.it/en/annunci/108314127/', 'https://www.immobiliare.it/en/annunci/109828553/', 'https://www.immobiliare.it/en/annunci/109610787/', 'https://www.immobiliare.it/en/annunci/109540027/', 'https://www.immobiliare.it/en/annunci/104387153/', 'https://www.immobiliare.it/en/annunci/107253629/', 'https://www.immobiliare.it/en/annunci/107254123/', 'https://www.immobiliare.it/en/annunci/109630733/', 'https://www.immobiliare.it/en/annunci/109467289/', 'https://www.immobiliare.it/en/annunci/101784767/', 'https://www.immobiliare.it/en/annunci/104392073/', 'https://www.immobiliare.it/en/annunci/109810011/', 'https://www.immobiliare.it/en/annunci/109425889/', 'https://www.immobiliare.it/en/annunci/109525277/', 'https://www.immobiliare.it/en/annunci/107724987/', 'https://www.immobiliare.it/en/annunci/109776901/', 'https://www.immobiliare.it/en/annunci/108099741/', 'https://ww

In [23]:
def dict_to_dataframe(appartment_information, dataframe_dict=dict()):

    if not dataframe_dict :
        dataframe_dict['city'] = list()
        dataframe_dict['street'] = list()
        dataframe_dict['district'] = list()
        dataframe_dict.update({key : list() for key in appartment_information['Features'].keys()})
        dataframe_dict.update({key : list() for key in appartment_information['Expenses'].keys()})
        dataframe_dict.update({key : list() for key in appartment_information['Energy efficiency'].keys()})
        
    # Decomposition of the address
    address = appartment_information['Address']
    city = address[0]
    street = address[1]
    district = address[2]
    
    dataframe_dict['city'] += [city]
    dataframe_dict['street'] += [street]
    dataframe_dict['district'] += [district]
    
    # Decomposition of the features
    features = appartment_information['Features']
    for key, value in features.items() :
        try :
            dataframe_dict[key] += [value]
        except KeyError :
            pass
    
    
    # Decomposition of the expenses
    expenses = appartment_information['Expenses']
    for key, value in expenses.items() :
        try :
            dataframe_dict[key] += [value]
        except KeyError :
            pass
    
    # Decomposition of the energy efficiency
    energy_efficiency = appartment_information['Energy efficiency']
    for key, value in energy_efficiency.items() :
        try :
            dataframe_dict[key] += [value]
        except KeyError :
            pass

    max_length = 0
    for value in dataframe_dict.values() :
        if len(value) > max_length :
            max_length = len(value)
            
    for key, value in dataframe_dict.items() :
        if len(value) < max_length :
            dataframe_dict[key] += [pd.NA]
    
    return dataframe_dict       

In [24]:
dataframe_dict = dict()
number_of_values = len(appartments.values())
for i, appartment in enumerate(appartments.values()) :
    dataframe_dict.update(dict_to_dataframe(appartment))
    print(f'Values left to process : {number_of_values - i}')

Values left to process : 2000
Values left to process : 1999
Values left to process : 1998
Values left to process : 1997
Values left to process : 1996
Values left to process : 1995
Values left to process : 1994
Values left to process : 1993
Values left to process : 1992
Values left to process : 1991
Values left to process : 1990
Values left to process : 1989
Values left to process : 1988
Values left to process : 1987
Values left to process : 1986
Values left to process : 1985
Values left to process : 1984
Values left to process : 1983
Values left to process : 1982
Values left to process : 1981
Values left to process : 1980
Values left to process : 1979
Values left to process : 1978
Values left to process : 1977
Values left to process : 1976
Values left to process : 1975
Values left to process : 1974
Values left to process : 1973
Values left to process : 1972
Values left to process : 1971
Values left to process : 1970
Values left to process : 1969
Values left to process : 1968
Values lef

In [30]:
dataframe = pd.DataFrame(dataframe_dict)
# dataframe = pd.read_pickle('appartments.pkl')
dataframe

,city,street,district,Reference and listing Date,contract,type,surface,rooms,floor,total building floors,...,availability,other features,price,condominium fees,deposit,year of construction,condition,heating,Air conditioner,Energy Efficiency
0,Milan,Via Federico Tesio,San Siro,a202406 - 02/26/2024,"Rent, 4+4",Apartment | Full ownership | Stately property ...,170 m² - See detail,"4 (3 bedrooms, 1 other), 2 bathrooms, kitchen ...","3°, with lift",4 floors,...,Available from 01/04/2024,Tennis courtElectric gateFireplaceOptic fiberS...,"€ 3,000/month",€ 500/month,"€ 9,000",1960,Excellent / Refurbished,"Central, with radiators, gas powered","Independent, cold/hot",D101.02 kWh/m² year
1,Milan,via MAURO MACCHI 27,Centrale,MACCHI 4P4 - 02/26/2024,"Rent, 4+4",Apartment | Full ownership | Stately property ...,150 m² - See detail,"4 (3 bedrooms, 1 other), 2 bathrooms, kitchen ...","4°, with lift",5 floors,...,Available from 01/04/2024,Optic fiberVideo entryphoneSecurity doorTerrac...,"€ 3,140/month",€ 417/month,"€ 37,680",1965,Excellent / Refurbished,"Central, with radiators, powered by methane","Independent, cold/hot",F170 kWh/m² year
2,Milan,via GIUSEPPE FRUA 21/6,Frua,VERCELLI HOME 4 - 02/10/2024,"Rent, 4+4",Apartment | Full ownership | Stately property ...,160 m² | commercial 176 m² - See detail,"4 (3 bedrooms, 1 other), 2 bathrooms, kitchen ...","2 floors: Ground floor, 4°, with lift",8 floors,...,Available,Electric gateOptic fiberVideo entryphoneSecuri...,"€ 3,290/month",€ 500/month,"€ 9,870",1970,New / Under construction,"Central, floor heating, powered by methane","Independent, cold/hot",F170 kWh/m² year
3,Milan,Piazza 25 Aprile,Garibaldi - Corso Como,Aprile - 03/01/2024,"Rent, 4+4",Apartment | Full ownership | Luxury property,100 m² - See detail,"3 (2 bedrooms, 1 other), 2 bathrooms, open kit...","4°, with lift, with disabled access",4 floors,...,Available,Electric gateOptic fiberVideo entryphoneAlarm ...,"€ 2,300/month",€ 200/month,€ 1,1900,Excellent / Refurbished,"Independent, floor heating, powered by lpg","Independent, cold/hot",G≥ 175 kWh/m² year
4,Milan,Via Alberto Nota,Bicocca,AM-Nota - 03/01/2024,"Rent, 4+4",Apartment | Full ownership | Stately property ...,50 m² - See detail,"2 (1 bedroom, 1 other), 1 bathroom, open kitchen",Mezzanine,4 floors,...,Available from 01/06/2024,Optic fiberSecurity doorExternal exposureFull ...,"€ 1,000/month",€ 200/month,Not indicated,1960,Good condition / Liveable,"Central, with radiators, powered by methane","Independent, cold/hot",F3.51 kWh/m² year
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Milan,Via Timavo,Maggiolina,via Timavo trilocale arredato - 06/27/2023,"Rent, 4+4",Apartment | Full ownership | Stately property ...,85 m² - See detail,"3 (2 bedrooms, 1 other), 2 bathrooms, semi-hab...",1°,2 floors,...,Available,Video entryphoneSecurity doorClosetBalconyCent...,"€ 1,900/month",€ 500/month,"€ 5,700",1970,Excellent / Refurbished,"Central, with radiators, powered by methane","Independent, cold/hot",G≥ 175 kWh/m² year
1996,Milan,Via Gian Carlo Castelbarco,Bocconi,A-585-AN-MC - 03/04/2024,"Rent, open",Apartment | Full ownership | Luxury property,120 m² - See detail,"3 (2 bedrooms, 1 other), 2 bathrooms, kitchen ...","2°, with lift",4 floors,...,Available,Electric gateOptic fiberVideo entryphoneSecuri...,"€ 2,500/month",€ 300/month,"€ 7,500",1900,Excellent / Refurbished,"Central, with radiators, powered by methane","Independent, cold/hot",D≥ 175 kWh/m² year
1997,Milan,Via Mauro Macchi,Centrale,65736 - 02/29/2024,"Rent, 4+4",Apartment | Full ownership | Stately property ...,50 m² - See detail,"2 (1 bedroom, 1 other), 1 bathroom, open kitchen","5°, with lift, with disabled access",6 floors,...,Available,Optic fiberVideo entryphoneSecurity doorExtern...,"€ 1,500/month",€ 150/month,Not indicated,2022,Good condition / Liveable,"Independent, with radiators, powered by methane","Independent, cold/hot",A272.68 kWh/m² year
1998,Milan,Piazza Guglielmo 

In [108]:
dataframe = pd.DataFrame(dataframe_dict)

# Edit prices column
dataframe = dataframe[dataframe['price']!="NA"]
dataframe = dataframe[dataframe['surface']!="NA"]
dataframe = dataframe[dataframe['price']!="price on application"]
dataframe['price']= dataframe['price'].str.replace("€ ","")
dataframe['price']= dataframe['price'].str.replace("/month","")
dataframe['price']= dataframe['price'].str.replace(",","").astype(float)
dataframe.drop(dataframe[dataframe['price'] >= 12000].index, inplace=True)
dataframe = dataframe.sort_values('price')

# Edit surfaces column
dataframe['surface'] = dataframe['surface'].str.replace(" m²","")
dataframe['surface'] = dataframe['surface'].str.extract(r'^(\d+)').astype(int)

# Edit floors column
dataframe['floor'] = dataframe['floor'].str.replace(r'^Ground floor.*$', '0', regex=True)
dataframe['floor'] = dataframe['floor'].str.replace(r'^Mezzanine.*$', '1', regex=True)
dataframe['floor'] = dataframe['floor'].str.replace(r'°.*$', "", regex=True)

# Edit district column
dataframe['district'] = dataframe['district'].str.replace(r"'", '', regex=True)

# Edit rooms column
dataframe['rooms'] = dataframe['rooms'].str.extract(r'^(\d+)')

# Edit street column
dataframe['street'] = dataframe['street'].str.replace(r"'", '', regex=True)
# dataframe['street'] = dataframe['street'].str.replace(r"^(€m²)*", 'N/A', regex=True)
dataframe['street'] = dataframe['street'].str.replace("m²", 'N/A')

dataframe.drop(columns=['Car parking'])
dataframe.replace(["N/A", "NA", "<NA>"], pd.NA, inplace=True)
dataframe.dropna(subset=['rooms', 'floor', 'district', 'street'], inplace=True)
dataframe = dataframe.rename(columns={"price": "price (k€)"})

In [109]:
# dataframe[['district', 'street' ,'surface', 'price (k€)', 'rooms', 'floor', 'condition', 'contract']]
dataframe[['district', 'street' ,'surface', 'price (k€)', 'rooms', 'floor']]
# dataframe

,district,street,surface,price (k€),rooms,floor
1561,Città Studi,"Via Francesco Lomonaco, 11, 20131 Milano MI, I...",100,400.0,2,1
923,Niguarda,Via Palanzone,66,500.0,3,4
1846,Casoretto,Via Eugenio Carpi,20,500.0,2,0
512,Dergano,Via Amilcare Bonomi,25,530.0,1,3
459,Città Studi,via BROCCHI,27,550.0,1,1
...,...,...,...,...,...,...
1294,Brera,Via dei Giardini,245,10000.0,5,1
1916,City Life,Via Ambrogio Spinola,234,10000.0,5,8
1123,Palestro,Via Gabrio Serbelloni,330,10000.0,5,7
784,Brera,Piazza SantErasmo,250,10000.0,5,1


In [106]:
dataframe.to_csv('appartments.csv')

In [107]:
dataframe.to_pickle('appartments.pkl')
# dataframe[['city', 'district', 'street' ,'surface', 'price (k€)', 'rooms', 'floor', 'condition', 'contract', 'URL']]